# N1Plotter

The **N1Plotter** is not derived directly from any **ROOT** class. Instead it provides a convenient possibility to create so-called *N-1* plots for cut-based sensitivity sudies.

Again we start by creating a test sample that we can play with:

In [ ]:
import ROOT
from mephisto import IOManager, N1Plotter

IOManager.CreateTestSample("testsample.root", tree="TestTree", nbranches=10, nevents=1e4)

Again we define the input and style of our dummy background and signal histograms:

In [ ]:
histoconfig = {
    # background histograms:
    "bkg1": {
        "weight": "0.6*branch_10",
        "template": "background",
        "fillcolor": "#6baff4",
        "title": "ttbar"
    },
    "bkg2": {
        "weight": "0.5*branch_10",
        "template": "background",
        "fillcolor": "#fff266",
        "title": "W+jets"
    },
    "bkg3": {
        "weight": "0.4*branch_10",
        "template": "background",
        "fillcolor": "#98ff79",
        "title": "Single top"
    },
    "bkg4": {
        "weight": "0.3*branch_10",
        "template": "background",
        "fillcolor": "#D182FF",
        "title": "Diboson"
    },
    "bkg5": {
        "weight": "0.2*branch_10",
        "template": "background",
        "fillcolor": "#ffc25e",
        "title": "Z+jets"
    },
    # signal histograms:
    "sig1": {
        "weight": "2.6*branch_8",
        "template": "signal",
        "linecolor": ROOT.kRed,
        "title": "Signal 1"
    },
    "sig2": {
        "weight": "2.5*branch_10",
        "template": "signal",
        "linecolor": ROOT.kBlue,
        "title": "Signal 2"
    },
}

Now to the interesting part: The N1Plotter will automatically create histograms for any cut expression in a list of cut expressions. Thus for each cut expression one also has to define an associated binning.

In [ ]:
# cuts can be defined as a list or a '&&'-separated string
# basic arithmetic operations are supported as well
mycuts = [
    "branch_4>1.5",
    "branch_1+branch_5<3.75",
    "(branch_6>=4.5)&&(abs(branch_7)<=9.5)",
]

mybinnings = {
    "branch_4": [20, 0.0, 10.0],  # fixed bin width
    "branch_1+branch_5": [20, 0.0, 10.0],
    "branch_6": [
        [i * 0.25 for i in range(40)] + [10, 11, 12, 13, 14, 15, 17.5, 20]
    ],  # variable bin width
    "abs(branch_7)": [10, 0.0, 10.0],
}

Next we create a instance of the N1Plotter and register our signal and background histograms to it! Note, that you need to specify if the registered histogram is of type ``signal`` or ``background`` (i.e. we can just use the name of the styling template from entries of the dictionary we defined above).

By default the [Asimov significance](http://www.pp.rhul.ac.uk/~cowan/stat/medsig/medsigNote.pdf) $Z_{A}$ is used as a sensitivity measure and the value is calculated for each bin via the **SensitivityScan** class. As it's a property of the N1Plotter class, you could also define a ``sensitivitymeasure`` yourself in the initialization step.

Finally you can save all *N-1* plots to the current directory with the ``Print`` method.

In [ ]:
n1plotter = N1Plotter(
    preselection=["branch_8>1.0"], # optional, no binning definitions required
    cuts=mycuts,
    binnings=mybinnings
)

for process, config in histoconfig.items():
    n1plotter.Register("testsample.root", tree="TestTree", type=config["template"], **config)

n1plotter.Print(".", logy=False, mkdir=True)

Let's see what we've got:

In [ ]:
!pdfunite N-1_*.pdf plots.pdf

from IPython.display import IFrame, display
display(IFrame("plots.pdf", width=600, height=450))  # you might need to use Firefox for this...

That's all there is for now. Bye!